In [33]:
### IMPORT REQUIRED LIBRARIES ###

# import re 
from datetime import datetime 
import calendar

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import geopandas as gpd


# Set option for pd dataframes to show all columns when called 
pd.set_option("display.max_columns", None)

In [20]:
# PROTESTS DATA 

# Read data 
protests_df = pd.read_csv("ccc_compiled.csv") 

# Mutate column types to requrement 
protests_df["date"] = pd.to_datetime(protests_df["date"])
protests_df["month_year"] = pd.to_datetime(protests_df['date']).dt.to_period('M').dt.to_timestamp()
protests_df["month_year"] = pd.to_datetime(protests_df["month_year"])

# # Convert fips_code from float to numeric - for joining with other tables 
# protests_df["fips_code"] = pd.to_numeric(protests_df["fips_code"])

## Filter dataset to get only required events
# After May 25th, size_cat>1, issues contains ("policing" | "racisim"), online != 1

# print(len(protests_df))
# Removing data collected before May 2020 
protests_df_filtered = protests_df[protests_df["date"] > datetime(2020, 1, 1)]
protests_df_filtered = protests_df_filtered[protests_df_filtered["date"] < datetime(2021, 1, 1)]
# print(len(protests_df_filtered))

# Filter for non-online issues only 
protests_df_filtered = protests_df_filtered[protests_df_filtered["online"] != 1]
# print(len(protests_df_filtered))

# Filter for issues where the issues included "policing" or "racism"
protests_df_filtered = protests_df_filtered[protests_df_filtered["issues"].str.contains("policing|racism", case=True, na=False)]
# print(len(protests_df_filtered))

# Filter where size_cat > 1; size_cat key: 0 = unknown; 1 = 1-99; 2 = 100-999; 3 = 1,000-9,999; 4 = 10,000+ 
protests_df_filtered = protests_df_filtered[protests_df_filtered["size_cat"] > 0]
# print(len(protests_df_filtered))

# Drop unrequired data. Check protests_df object for full data 
protests_df_filtered = protests_df_filtered.drop(protests_df_filtered.iloc[:, 6:13], axis = 1)
protests_df_filtered = protests_df_filtered.drop(['size_text'], axis = 1)
protests_df_filtered = protests_df_filtered.drop(protests_df_filtered.iloc[:, 13:53], axis = 1)

# Update the "resolved_county" column to match the format of the "county" column - remove characters "county "
protests_df_filtered["resolved_county"] = pd.Series(protests_df_filtered["resolved_county"]).str.replace(' County', '', regex=True)

# Remove data with missing fips codes (5 records at this point)
protests_df_filtered = protests_df_filtered.dropna(subset=["fips_code"], axis = 0)


# protests_df_filtered.head()

/Users/aymanbari/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,8,9,11,12,23,25,27,30,31,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
# protests_df[["organizations", "participants"]].head(100)
protests_df.head()

,date,locality,state,location,location_detail,county,online,type,title,macroevent,actors,organizations,participants,claims,valence,issues,size_text,size_low,size_high,size_mean,size_cat,arrests,arrests_any,injuries_crowd,injuries_crowd_any,injuries_police,injuries_police_any,property_damage,property_damage_any,chemical_agents,participant_measures,police_measures,participant_deaths,police_deaths,source_1,source_2,source_3,source_4,source_5,source_6,source_7,source_8,source_9,source_10,source_11,source_12,source_13,source_14,source_15,source_16,source_17,source_18,source_19,source_20,source_21,source_22,source_23,source_24,source_25,source_26,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code,month_year
0,2017-01-20,Seattle,WA,"Seattle, WA",University of Washington campus,NaN,0.0,protest; demonstration,NaN,NaN,general protestors,NaN,NaN,anti-Yiannopolous; anti-Breitbart; anti-Trump,1.0,executive,NaN,200.0,700.0,450.0,2,1,1,1,1,0,0,1.0,1,NaN,NaN,NaN,NaN,NaN,http://infoweb.newsbank.com.du.idm.oclc.org/re...,http://www.seattletimes.com/seattle-news/north...,http://www.seattletimes.com/seattle-news/educa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.606209,-122.332071,Seattle,King County,WA,53033.0,2017-01-01
1,2017-01-21,Adak,AK,"Adak, AK",NaN,NaN,0.0,march,NaN,20170121-womensmarch,NaN,NaN,NaN,women's rights are human rights,1.0,women's rights,NaN,10.0,10.0,10.0,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,adn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.880000,-176.658056,Adak,Aleutians West,AK,2016.0,2017-01-01
2,2017-01-21,Anchorage,AK,"Anchorage, AK",NaN,NaN,0.0,march,NaN,20170121-womensmarch,NaN,NaN,NaN,women's rights are human rights,1.0,women's rights,NaN,2000.0,4000.0,3000.0,3,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,https://www.adn.com/alaska-news/2017/01/21/lar...,https://twitter.com/julia_omalley/status/82290...,http://www.ktuu.com/content/news/Thousands-of-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.218056,-149.900278,Anchorage,Anchorage,AK,2020.0,2017-01-01
3,2017-01-21,Bethel,AK,"Bethel, AK",NaN,NaN,0.0,march,NaN,20170121-womensmarch,NaN,NaN,NaN,women's rights are human rights,1.0,women's rights,NaN,40.0,100.0,70.0,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,https://www.adn.com/slideshow/visual/2017/01/2...,http://kyuk.org/post/slideshow-bethel-proteste...,https://jackpineradicals.com/boards/topic/wome...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.792222,-161.755833,Bethel,Bethel,AK,2050.0,2017-01-01
4,2017-01-21,Cordova,AK,"Cordova, AK",NaN,NaN,0.0,march,NaN,20170121-womensmarch,NaN,NaN,NaN,women's rights are human rights,1.0,women's rights,NaN,108.0,114.0,111.0,2,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,onsite eyewitness who did headcount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.542409,-145.752541,Cordova,Valdez-Cordova,AK,2261.0,2017-01-01


In [21]:
# COUNTY POPULATION DATA 

# Read data 
county_pop_df = pd.read_csv(
    "population_by_county/DECENNIALPL2020.P1_data_with_overlays_2021-11-21T135424.csv", 
    header = 1, 
) 

# Remove unrequried fields - remove population by race. Keep total only. 
county_pop_df_filtered = county_pop_df.drop(county_pop_df.iloc[:,3:], axis = 1)

# Remame columns 
county_pop_df_filtered = county_pop_df_filtered.set_axis(['fips_code', 'county', 'population'], axis = 1)

# Mutate fips_code column for join with protests dataset 
county_pop_df_filtered["fips_code"] = county_pop_df_filtered["fips_code"].apply(lambda x: float(x[9:]))


In [22]:
# DATA FRAME FOR TOTAL STATE POPULATION 

# Source: https://data.ers.usda.gov/reports.aspx?ID=17827 

# Read data 
state_pop_df = pd.read_csv("StatePopulationReport.csv", header = 2)

# Frop columns that are not required 
state_pop_df = state_pop_df.drop(state_pop_df.iloc[:, 1:4], axis = 1)
state_pop_df = state_pop_df.drop(state_pop_df.iloc[:, 2:], axis = 1)

# Rename columns for join 
state_pop_df = state_pop_df.rename(columns={'Name': 'state_name', 'Pop. 2020': 'state_population'})
# Remove commas to convert to float 
state_pop_df["state_population"] = pd.Series(state_pop_df["state_population"]).str.replace(',', '', regex=True)
# Convert from string to float 
state_pop_df["state_population"] = pd.to_numeric(state_pop_df["state_population"], downcast ="float")

# Check df 
state_pop_df.head()

,state_name,state_population
0,United States,331449280.0
1,Alabama,5024279.0
2,Alaska,733391.0
3,Arizona,7151502.0
4,Arkansas,3011524.0


In [6]:
# # COUNTY GEO SHAPE DATA 

# # Load in data 
# geo_df = gpd.read_file("us-county-boundaries/us-county-boundaries.shp")

# # Add a column called fips code - state and county codes for joining with main dataset 
# geo_df["fips_code"] = pd.to_numeric((geo_df["statefp"] + geo_df["countyfp"]), downcast='float')

# # geo_df.head()

In [25]:
# JOIN PROTESTS, COUNTY POPULATION AND GEO SHAPE DATA 

df = pd.merge(protests_df_filtered, county_pop_df_filtered, on='fips_code', how='left')

# Despite having the fips codes, the protests dataset is missing the origin county for some records. 
# Updating "county_y" from the census dataset to match the format of "county_resolved" for comparison 
df["county_y"] = df["county_y"].str.replace(' County([\s\S]*)$', '', regex=True)

# Add a column to account for the county population 
df["size_mean_over_pop"] = df["size_mean"]/df["population"]

# # Merge with geo data 
# df = pd.merge(df, geo_df, on='fips_code', how='left')

# # Merge with geo data 
# df = pd.merge(df, state_pop_df, on='state_name', how='left')

# # Add a column to account for the state population 
# df["size_mean_over_state_pop"] = df["size_mean"]/df["state_population"]

# # Add a column to indicate if county and resolved_county are the same 
# df["single_county"] = (df["county_y"] == df["resolved_county"])

# # Check if merge worked as expected 
# print(len(df)) 
# print(len(protests_df_filtered))
# print(len(df) == len(protests_df_filtered))

In [26]:
df.head()

,date,locality,state,location,location_detail,county_x,claims,valence,issues,size_low,size_high,size_mean,size_cat,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code,month_year,county_y,population,size_mean_over_pop
0,2020-01-05,Joplin,MO,"Joplin, MO",Joplin police headquarters,Jasper,"for justice for David Ingle, against police br...",1.0,healthcare; policing,15.0,20.0,18.0,1,37.084227,-94.513281,Joplin,Jasper,MO,29097.0,2020-01-01,Jasper,122761.0,0.000147
1,2020-01-06,Waukegan,IL,"Waukegan, IL",City Hall,Lake,"against police brutality, against racism",1.0,policing; racism,20.0,20.0,20.0,1,42.363633,-87.844794,Waukegan,Lake,IL,17097.0,2020-01-01,Lake,714342.0,0.000028
2,2020-01-08,Richmond,VA,"Richmond, VA",Virginia State Capitol,Henrico,for removal of Confederate monuments,1.0,racism,93.0,93.0,93.0,1,37.540725,-77.436048,Richmond,NaN,VA,51760.0,2020-01-01,"Richmond city, Virginia",226610.0,0.000410
3,2020-01-10,Los Angeles,CA,"Los Angeles, CA",Watts,Los Angeles,"against presidential candidate Pete Buttigieg,...",1.0,executive; policing; racism,12.0,12.0,12.0,1,34.052234,-118.243685,Los Angeles,Los Angeles,CA,6037.0,2020-01-01,Los Angeles,10014009.0,0.000001
4,2020-01-10,Barberton,OH,"Barberton, OH",Barberton High School,Summit,"against police brutality, against racism",1.0,policing; racism,2.0,2.0,2.0,1,41.012833,-81.605122,Barberton,Summit,OH,39153.0,2020-01-01,Summit,540428.0,0.000004


In [32]:
# DATA FRAME with VALENCE = 2 ONLY 
# df.head()
df_valence_1 = df[df["valence"] == 1.0]
df_valence_2 = df[df["valence"] == 2.0]

df_valence_2.head()

df_valence_1.to_csv("df_valence_1.csv", sep=",")
df_valence_2.to_csv("df_valence_2.csv", sep=",")

# # Sanity check 
# df_valence_2["valence"].unique()
# print(len(df))
# print(len(df_valence_1))
# print(len(df_valence_2))
